In [ ]:
# Important Packages
!pip install nest-asyncio
!git clone --depth=1 https://github.com/twintproject/twint.git
!cd twint && pip3 install . -r requirements.txt
!pip install langdetect
!pip install -q transformers

In [55]:
# Dependencies & Libraries
import nest_asyncio
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from langdetect import detect
import os
import re

BASEDIR = os.getcwd()
print(BASEDIR)
nest_asyncio.apply()


d:\projects\sentic-trading-bot\rnn_bot


In [96]:
# Search Parameters
today = datetime.today().strftime("%Y-%m-%d")
tomorrow = (datetime.today() + timedelta(1)).strftime("%Y-%m-%d")
past_seven_days = (datetime.today() - timedelta(7)).strftime("%Y-%m-%d")

print(f"Getting tweets ranging from\n{today} to {tomorrow}")

keywords = ["BTC", "BITCOIN", "BTC-USD"]

2021-09-16
Getting tweets ranging from
2021-09-23 to 2021-09-24


In [101]:
# Data Collection
import twint

# tweets = pd.DataFrame(columns=["date", "user_id", "username", "tweet", "nlikes", "nretweets", "nreplies"])
tweets = pd.DataFrame(columns=["date","tweet"])
c = twint.Config()
# c.Username = "elonmusk"
# c.Limit = 10
c.Limit = 10000
c.Store_csv = False
c.Pandas = True
c.Hide_output = True
c.Since = today
c.Until = tomorrow
c.Min_likes = 10
# Getting only English tweets, undefined == all, 
# Lang codes here https://github.com/twintproject/twint/wiki/Langauge-codes
# This shit is not accurate
c.Lang = "en"

c.Search = "btc OR bitcoin"

twint.run.Search(c)

df = twint.storage.panda.Tweets_df
try:
    #tweets = tweets.append(df[["date", "user_id", "username", "tweet", "nlikes", "nretweets", "nreplies"]])
    tweets = tweets.append(df[["date","tweet"]])
except KeyError:
    print("No more data!")

tweets.to_csv(f"{BASEDIR}/data/tweets.csv")
tweets.head()

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


,date,tweet
0,2021-09-21 11:00:02,Half of Brazilians would welcome BTC with open...
1,2021-09-21 10:59:33,Genocide Via Medicine via @maxigan @DollarVigi...
2,2021-09-21 10:59:09,Multiple bitcoiners expressed concern to me to...
3,2021-09-21 10:59:08,"😱BITCOIN CAYENDO EN VIVO, VEAMOS ESTO /V797 h..."
4,2021-09-21 10:58:55,"Current position: $9,916,500 UPnL: -22 #btc Mo..."


In [106]:
# Cleanining Functions
def clean(tweet):
    try:
        if detect(tweet) != "en":
            return np.NaN
    except:
        return np.NaN
    tweet = re.sub("@[A-Za-z0-9_]+","", tweet)
    tweet = re.sub("#[A-Za-z0-9_]+","", tweet)
    tweet = re.sub(r"(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)", "", tweet)
    tweet = re.sub(r"www.\S+", "", tweet)
    return tweet

In [107]:
tweets["cleaned_tweet"] = tweets["tweet"].apply(clean)
print(tweets.head())
tweets = tweets.dropna()
tweets = tweets.drop_duplicates()
tweets.to_csv(f"{BASEDIR}/data/tweets_cleaned.csv")

                  date                                              tweet  \
0  2021-09-21 11:00:02  Half of Brazilians would welcome BTC with open...   
1  2021-09-21 10:59:33  Genocide Via Medicine via @maxigan @DollarVigi...   
2  2021-09-21 10:59:09  Multiple bitcoiners expressed concern to me to...   
3  2021-09-21 10:59:08  😱BITCOIN CAYENDO EN VIVO, VEAMOS ESTO /V797  h...   
4  2021-09-21 10:58:55  Current position: $9,916,500 UPnL: -22 #btc Mo...   

                                       cleaned_tweet  
0  Half of Brazilians would welcome BTC with open...  
1  Genocide Via Medicine via      🏴‍☠️ $ARRR   $X...  
2  Multiple bitcoiners expressed concern to me to...  
3                                                NaN  
4  Current position: $9,916,500 UPnL: -22  Mood: ...  


In [108]:
# Sentiment Analysis
from transformers import pipeline

sentiment = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are 

In [113]:
def sentiment_analysis(tweet):
    label, score = sentiment(tweet)[0].values()
    if label == "NEGATIVE":
        return 1 - score
    else:
        return score

tweets["sentiment"] = tweets["tweet"].apply(sentiment_analysis)
tweets.head()
tweets.to_csv(f"{BASEDIR}/data/tweets_sentiment.csv")

In [115]:
# Average Sentiment Score
print("Average Sentiment:", tweets["sentiment"].sum()/len(tweets.index))


Average Sentiment: -0.607606787152969
